In [1]:
# conda install pymysql

In [2]:
# import pymysql.cursors
# import pandas as pd
# connection = pymysql.connect(host='ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com', 
#                              user='read_only', password='WHC63fKyjudfSz2U', db='CAPC_APIGATEWAY',
#                              cursorclass=pymysql.cursors.SSDictCursor)

In [3]:
import re

In [4]:
import os

# folder path
dir_path = r'/home/trishalaswain/Downloads/JanSCP18/JanSCP-New18v3Selected/'

# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)) and not str(path).endswith("HCPCS.tsv") and not str(path).endswith("CPT.tsv") and not str(path).endswith("CM.tsv")  and not str(path).endswith("EANDM.tsv"):

        # print(path)
        
        res.append(dir_path + path)
        # print(path)

FileNotFoundError: [Errno 2] No such file or directory: '/home/trishalaswain/Downloads/JanSCP18/JanSCP-New18v3Selected/'

In [ ]:
df_final = pd.DataFrame()

In [5]:
def initialize(index):
    df_final.at[index,'1_doc_only_counts'] = 0
    df_final.at[index,'2_doc_only_counts'] = 0
    df_final.at[index,'3_doc_only_counts'] = 0
    df_final.at[index,'4_doc_only_counts'] = 0
    df_final.at[index,'other_no_of_docs_counts'] = 0

In [25]:
# df_final['facility_code'] = None
df_final['facility_name'] = None
# df_final['prod_id'] = None
df_final['no_of_encounters'] = None
df_final['total_docs'] = None
df_final['doc_mean'] = 0
df_final['doc_std_dev'] = 0
df_final['25_percentile'] = 0
df_final['50_percentile'] = 0
df_final['75_percentile'] = 0
df_final['min_doc_count'] = 0
df_final['max_doc_count'] = 0

In [26]:
def fill_df_final(index,facility_name,total_no_of_encounters,total_no_of_docs):
    df_final.at[index,'facility_name'] = facility_name
    # df_final.at[index,'individual_doc_counts'] = docs
    df_final.at[index,'no_of_encounters'] = total_no_of_encounters
    df_final.at[index,'total_docs'] = total_no_of_docs
    initialize(index)

In [27]:
# df = pd.read_excel('/home/trishalaswain/Downloads/details.xlsx')

In [28]:
for file in res:
    
    filename = re.findall("SCP-\d\d.*", file)
    # print(str(filename))
    df = pd.read_table(file, index_col = False)
    for index,row in df.iterrows():
        df1 = df['encDocIds'].dropna
        df1.shape
        index = 0
        for index, r in df1.iterrows():
            
            
        

In [9]:
# def fill_stats(index,sql_

In [35]:
# df1.dropna('encDocIds').shape

/tmp/ipykernel_10884/1506703620.py:1: FutureWarning: In a future version of pandas all arguments of Series.dropna will be keyword-only.
  df1.dropna('encDocIds').shape


ValueError: No axis named encDocIds for object type Series

In [36]:
df1

0                     NaN
1                     NaN
2                     NaN
3                     NaN
4                     NaN
              ...        
494              26524988
495              26685627
496              26524448
497              25727297
498    26085730, 26803037
Name: encDocIds, Length: 499, dtype: object

In [10]:
index = 0

for index,row in df.iterrows():
    # filename = str(row['facility_code']) + "_" + str(row['facility_name']) + "_" + str(row['prod_id']) +  ".xlsx"

    sql = """SELECT 
        em.id AS encounter_id,
        am.account_number,
        COUNT(dm.id) AS document_count
    FROM
        encounter_mst em
            INNER JOIN
        account_mst am ON am.id = em.account_id
            INNER JOIN
        document_mst dm ON dm.account_id = am.id
    WHERE
        is_physician_document = 1
            AND am.facility_id = """  + str(row['prod_id']) + """
            AND dm.created_date > '2023-01-01'
    GROUP BY account_number
    ORDER BY document_count DESC;"""
    
    
    sql_query = pd.read_sql(sql,con = connection)
    total_no_of_encounters = sql_query['encounter_id'].count()
    total_no_of_docs = sql_query['document_count'].sum()
    fill_df_final(index, row['facility_code'], row['facility_name'], row['prod_id'], total_no_of_encounters, total_no_of_docs)
    
        
#   Finding count of encounters with only1,only2,only3,only4 & more than4 docs
    for doc_counts in sql_query.document_count:
        if doc_counts == 1:
            df_final.at[index,'1_doc_only_counts'] += 1  
            # print(type(df_final.at[index,'1_doc_only_counts']))
        if doc_counts == 2:
            df_final.at[index,'2_doc_only_counts'] += 1   
        if doc_counts == 3:
            df_final.at[index,'3_doc_only_counts'] += 1   
        if doc_counts == 4:
            df_final.at[index,'4_doc_only_counts'] += 1   
        else:
            df_final.at[index,'other_no_of_docs_counts'] += 1  
            
    sql_query = sql_query.sort_values(by=['document_count'], ascending=True).reset_index()
    
    #   Finding mean of doc_counts
    mean = sql_query['document_count'].mean()
    df_final.at[index,'doc_mean'] = mean

    #   Finding standard deviation of doc_counts
    std_dev = sql_query['document_count'].std()
    df_final.at[index,'doc_std_dev'] = std_dev

    #   Finding 25th percentile
    _25_percentile = sql_query['document_count'].quantile(0.25)
    df_final.at[index,'25_percentile'] = _25_percentile

    #   Finding 50th percentile
    _50_percentile = sql_query['document_count'].quantile(0.50)
    df_final.at[index,'50_percentile'] = _50_percentile

    #   Finding 75th percentile
    _75_percentile = sql_query['document_count'].quantile(0.75)
    df_final.at[index,'75_percentile'] = _75_percentile

    #   Finding minimum doc_counts of all
    minimum = sql_query['document_count'].min()
    df_final.at[index,'min_doc_count'] = minimum

    #   Finding maximum doc_counts of all
    maximum = sql_query['document_count'].max()
    df_final.at[index,'max_doc_count'] = maximum

    columns_titles = ['facility_code','facility_name','prod_id','no_of_encounters','total_docs','1_doc_only_counts','2_doc_only_counts'
                      ,'3_doc_only_counts','4_doc_only_counts','other_no_of_docs_counts','doc_mean','doc_std_dev','25_percentile','50_percentile'
                      ,'75_percentile','min_doc_count','max_doc_count']
      
    df_final = df_final.reindex(columns=columns_titles)

    index = index + 1
df_final.to_excel("/home/trishalaswain/Downloads/Data/document_stats_report.xlsx",index = False)
print("created successfully")

/home/trishalaswain/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/trishalaswain/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/trishalaswain/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/trishalaswain/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy 

created successfully


In [11]:
df_final

,facility_code,facility_name,prod_id,no_of_encounters,total_docs,1_doc_only_counts,2_doc_only_counts,3_doc_only_counts,4_doc_only_counts,other_no_of_docs_counts,doc_mean,doc_std_dev,25_percentile,50_percentile,75_percentile,min_doc_count,max_doc_count
0,2336,HARLAN ARH HOSPITAL,108,1181,3533,957.0,115.0,40.0,14.0,1167.0,2.991533,16.269445,1.0,1.0,1.0,1.0,385.0
1,2464,DEQUINCY MEMORIAL HOSPITAL,365,287,989,37.0,73.0,76.0,46.0,241.0,3.445993,2.766503,2.0,3.0,4.0,1.0,33.0
2,4035,PLAINS REGIONAL MEDICAL CENTER,331,2330,6047,368.0,909.0,641.0,222.0,2108.0,2.595279,1.239355,2.0,2.0,3.0,1.0,9.0
3,41,WINN PARISH MEDICAL CENTER,53,664,1813,116.0,307.0,126.0,48.0,616.0,2.730422,2.221771,2.0,2.0,3.0,1.0,22.0
4,4044,DR. DAN C. TRIGG MEMORIAL HOSPITAL-ICP,259,442,1033,88.0,223.0,76.0,31.0,411.0,2.337104,1.255168,2.0,2.0,3.0,1.0,11.0
5,1676,MERCYONE WATERLOO MEDICAL CENTER,166,2050,4164,72.0,1874.0,74.0,28.0,2022.0,2.031220,0.365684,2.0,2.0,2.0,1.0,5.0
6,5542,CHRISTUS SPOHN HOSPITAL CORPUS CHRISTI - SHORE...,81,5519,8313,3713.0,1358.0,283.0,104.0,5415.0,1.506251,1.945969,1.0,1.0,2.0,1.0,116.0
7,2341,MARY BRECKINRIDGE ARH HOSPITAL,114,414,582,320.0,49.0,27.0,9.0,405.0,1.405797,0.896355,1.0,1.0,1.0,1.0,6.0
8,4525,OHIOHEALTH MANSFIELD HOSPITAL,264,3193,4829,1588.0,1574.0,31.0,0.0,3193.0,1.512371,0.518988,1.0,2.0,2.0,1.0,3.0
9,493430,OHIOHEALTH EMERGENCY CARE-ASHLAND,322,868,1309,427.0,441.0,0.0,0.0,868.0,1.508065,0.500223,1.0,2.0,2.0,1.0,2.0


In [12]:
# sql_query['document_count'].describe(percentiles=0.25)

In [13]:
sql_query['encounter_id'].unique().size


379

In [14]:
sql_query.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           379 non-null    int64 
 1   encounter_id    379 non-null    int64 
 2   account_number  379 non-null    object
 3   document_count  379 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 12.0+ KB
